In [ ]:
#count all the words
from pyspark import SparkContext

# Load the text file into an RDD
projec_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/project.txt")

# Split the lines into words, filter out empty strings, and count each word
word_counts_rdd = (projec_rdd.flatMap(lambda line: line.split(" "))
                  .filter(lambda word: word != "")  # filter out empty strings
                  .map(lambda word: (word, 1))
                  .reduceByKey(lambda a, b: a + b)
                  .sortBy(lambda pair: pair[1], ascending=False))

# Collect and display the results
for word, count in word_counts_rdd.collect():
    print(f"{word}: {count}")


the: 26393
of: 15269
and: 11197
to: 8931
in: 7212
a: 6236
was: 3860
that: 2877
with: 2607
by: 2527
his: 2442
for: 2357
as: 2341
The: 2336
he: 2141
had: 2038
on: 1947
were: 1913
it: 1782
is: 1754
be: 1750
at: 1724
not: 1621
from: 1612
their: 1440
or: 1431
which: 1234
an: 1079
they: 1060
but: 1051
all: 1036
this: 906
her: 834
who: 819
In: 808
I: 780
one: 716
are: 712
been: 711
you: 699
have: 690
into: 690
said: 650
American: 644
him: 617
so: 588
more: 581
its: 577
He: 524
up: 512
Prince: 511
no: 507
them: 502
could: 500
It: 491
out: 474
if: 469
United: 467
would: 452
did: 449
she: 439
such: 436
other: 435
under: 434
New: 432
there: 431
upon: 431
when: 430
only: 423
than: 416
any: 403
may: 394
will: 393
made: 393
what: 379
against: 371
over: 371
=The: 369
government: 366
shall: 366
some: 348
about: 338
first: 337
after: 335
time: 334
has: 331
old: 322
new: 321
my: 315
two: 313
before: 311
same: 310
Congress: 307
pp.: 306
They: 305
men: 304
should: 300
our: 293
great: 288
French: 285
state

In [ ]:
#count specific words
import re

# List of words to track
target_words = ["america", "president", "washington"]

# Load the text file into an RDD
project_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/project.txt")

# Preprocess the text: Convert to lowercase, remove punctuation, and split by spaces
def preprocess(line):
    # Convert to lowercase
    line = line.lower()
    # Remove punctuation
    line = re.sub(r'[^a-zA-Z\s]', '', line)
    return line.split()

word_counts_rdd = (project_rdd.flatMap(preprocess)
                  .filter(lambda word: word in target_words) 
                  .map(lambda word: (word, 1))
                  .reduceByKey(lambda a, b: a + b))

# Display the results
for word, count in word_counts_rdd.collect():
    print(f"{word}: {count}")


america: 266
washington: 150
president: 326


In [ ]:
#top10 words
import re

# Load the text file into an RDD
project_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/project.txt")

# Preprocess the text: Convert to lowercase, remove punctuation, and split by spaces
def preprocess(line):
    # Convert to lowercase
    line = line.lower()
    # Remove punctuation
    line = re.sub(r'[^a-zA-Z\s]', '', line)
    return line.split()

word_counts_rdd = (project_rdd.flatMap(preprocess)
                  .map(lambda word: (word, 1))
                  .reduceByKey(lambda a, b: a + b))

# Order by Count in descending order and pick top 10
top_10_words_rdd = word_counts_rdd.takeOrdered(10, key=lambda x: -x[1])

# Display the top 10 words
for word, count in top_10_words_rdd:
    print(f"{word}: {count}")


the: 29772
of: 15465
and: 11712
to: 9204
in: 8119
a: 6626
was: 3946
that: 3075
he: 2785
it: 2740


In [ ]:
#avg temperature continent
from pyspark import SparkContext

# Load the text file into an RDD
project_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/city_temperature.csv")

# Extract the header
header = project_rdd.first()
project_data = project_rdd.filter(lambda line: line != header)

# Get the index of necessary fields from the header
region_index = header.split(",").index("Region")
avg_temp_index = header.split(",").index("AvgTemperature")

# Filter out invalid temperature entries and map data
def extract_region_avg_temp(line):
    values = line.split(",")
    return (values[region_index], float(values[avg_temp_index]))

filtered_data = project_data.filter(lambda line: "-99" not in line)
region_avg_temp_rdd = filtered_data.map(extract_region_avg_temp)

# Compute the average temperature for each region
sum_counts = region_avg_temp_rdd.combineByKey(lambda value: (value, 1),
                                              lambda x, value: (x[0] + value, x[1] + 1),
                                              lambda x, y: (x[0] + y[0], x[1] + y[1]))

average_by_region = sum_counts.mapValues(lambda v: v[0] / v[1])

# Display the results
for region, avg_temp in average_by_region.collect():
    print(f"Region: {region}, Average Temperature: {avg_temp:.2f}")


Region: Asia, Average Temperature: 68.11
Region: Africa, Average Temperature: 74.40
Region: Australia/South Pacific, Average Temperature: 62.30
Region: Europe, Average Temperature: 51.95
Region: Middle East, Average Temperature: 73.84
Region: North America, Average Temperature: 56.15
Region: South/Central America & Carribean, Average Temperature: 72.20


In [ ]:
#Year Average Temperature
from pyspark import SparkContext

# Load the text file into an RDD
project_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/city_temperature.csv")

# Extract the header and its indices
header = project_rdd.first()
year_index = header.split(",").index("Year")
region_index = header.split(",").index("Region")
avg_temp_index = header.split(",").index("AvgTemperature")
project_data = project_rdd.filter(lambda line: line != header)

# Filter rows based on valid AvgTemperature and "Asia" region
def valid_temperature_in_asia(line):
    values = line.split(",")
    try:
        avg_temp = float(values[avg_temp_index])
        region = values[region_index]
        return avg_temp != -99.0 and region == "Asia"
    except:
        return False

# Extract and map the necessary fields
project_processed = project_data.filter(valid_temperature_in_asia)\
                        .map(lambda line: (line.split(",")[year_index], float(line.split(",")[avg_temp_index])))

# Compute average temperature for each year
sum_counts = project_processed.combineByKey(lambda value: (value, 1),
                                        lambda x, value: (x[0] + value, x[1] + 1),
                                        lambda x, y: (x[0] + y[0], x[1] + y[1]))

average_by_year = sum_counts.mapValues(lambda v: v[0] / v[1])

# Sort the results by year and display
sorted_results = average_by_year.sortByKey().collect()

for year, avg_temp in sorted_results:
    print(f"Year: {year}, Average Temperature: {avg_temp:.2f}")

Year: 1995, Average Temperature: 67.26
Year: 1996, Average Temperature: 67.87
Year: 1997, Average Temperature: 68.35
Year: 1998, Average Temperature: 69.06
Year: 1999, Average Temperature: 68.03
Year: 2000, Average Temperature: 67.72
Year: 2001, Average Temperature: 67.74
Year: 2002, Average Temperature: 68.06
Year: 2003, Average Temperature: 67.91
Year: 2004, Average Temperature: 68.31
Year: 2005, Average Temperature: 67.78
Year: 2006, Average Temperature: 68.71
Year: 2007, Average Temperature: 68.87
Year: 2008, Average Temperature: 68.09
Year: 2009, Average Temperature: 68.73
Year: 2010, Average Temperature: 68.82
Year: 2011, Average Temperature: 67.65
Year: 2012, Average Temperature: 67.24
Year: 2013, Average Temperature: 67.84
Year: 2014, Average Temperature: 67.81
Year: 2015, Average Temperature: 68.67
Year: 2016, Average Temperature: 68.85
Year: 2017, Average Temperature: 68.46
Year: 2018, Average Temperature: 68.46
Year: 2019, Average Temperature: 68.67
Year: 2020, Average Tempe

In [ ]:
#City, County, Average
from pyspark import SparkContext

# Load the text file into an RDD
project_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/city_temperature.csv")

# Extract the header and determine the indices for required columns
header = project_rdd.first()
country_index = header.split(",").index("Country")
city_index = header.split(",").index("City")
avg_temp_index = header.split(",").index("AvgTemperature")
project_data = project_rdd.filter(lambda line: line != header)

# Filter rows based on the "Spain" country and valid AvgTemperature entries
def filter_spain_and_valid_temp(line):
    values = line.split(",")
    try:
        avg_temp = float(values[avg_temp_index])
        country = values[country_index]
        return country == "Spain" and avg_temp != -99.0
    except:
        return False

# Extract and map the fields (City, AvgTemperature)
project_processed = project_data.filter(filter_spain_and_valid_temp)\
                        .map(lambda line: (line.split(",")[city_index], float(line.split(",")[avg_temp_index])))

# Compute average temperature for each city in Spain
sum_counts = project_processed.combineByKey(lambda value: (value, 1),
                                        lambda x, value: (x[0] + value, x[1] + 1),
                                        lambda x, y: (x[0] + y[0], x[1] + y[1]))

average_by_city = sum_counts.mapValues(lambda v: v[0] / v[1])

# Display the results
for city, avg_temp in average_by_city.collect():
    print(f"City: {city}, Average Temperature: {avg_temp:.2f}")


City: Barcelona, Average Temperature: 61.79
City: Bilbao, Average Temperature: 59.17
City: Madrid, Average Temperature: 58.92


In [ ]:
#Country Capital City Temperature
import re

# Load the datasets into RDDs
cityTemp_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/city_temperature.csv")
capitals_rdd = sc.textFile("/FileStore/shared_uploads/hyeongjinjoo82@gmail.com/country_list.csv")

# Define a function to clean up and extract city names
def extract_city_names(line):
    cleaned_line = re.sub(r'"', '', line)  # remove quotes
    return cleaned_line.split(",")[1]

# Process the capitals dataset to get a list of capital cities
capital_cities = capitals_rdd.map(extract_city_names).collect()

# Broadcast the capital cities list
broadcasted_capitals = sc.broadcast(capital_cities)

# Define a function to process temperature data lines
def process_temperature_line(line):
    columns = line.split(",")
    # Filter for rows which aren't headers and where the city is a capital
    if columns[0] != 'Region' and columns[3] in broadcasted_capitals.value:
        return True
    return False

# Process the temperature dataset
filtered_data = cityTemp_rdd.filter(process_temperature_line)

# Extract relevant data: (Country, City) as the key and temperature as the value
city_temperature_data = filtered_data.filter(lambda x: float(x.split(",")[7]) != -99) \
                                     .map(lambda x: ((x.split(",")[1], x.split(",")[3]), float(x.split(",")[7])))

# Compute the sum and count of temperatures for each (Country, City)
aggregate_data = city_temperature_data.mapValues(lambda temp: (temp, 1)) \
                                      .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# Compute the average temperature for each (Country, City)
average_temperatures = aggregate_data.mapValues(lambda v: v[0] / v[1]) \
                                     .sortByKey()

average_temperatures.collect()


Out[14]: [(('Albania', 'Tirana'), 61.135237970711394),
 (('Algeria', 'Algiers'), 64.37253818654507),
 (('Argentina', 'Buenos Aires'), 62.91649875419788),
 (('Australia', 'Canberra'), 56.4014755343388),
 (('Austria', 'Vienna'), 51.46943722943731),
 (('Bahamas', 'Nassau'), 78.56622639449847),
 (('Bahrain', 'Manama'), 81.21907752273744),
 (('Bangladesh', 'Dhaka'), 78.72032520325234),
 (('Barbados', 'Bridgetown'), 81.0710863576477),
 (('Belarus', 'Minsk'), 45.39047039291643),
 (('Belgium', 'Brussels'), 51.5118640398354),
 (('Bermuda', 'Hamilton'), 71.59983807124873),
 (('Bolivia', 'La Paz'), 45.50655826558265),
 (('Bulgaria', 'Sofia'), 51.6243039116221),
 (('Burundi', 'Bujumbura'), 73.68981900452471),
 (('Canada', 'Ottawa'), 44.22525751375754),
 (('Central African Republic', 'Bangui'), 78.82184718529616),
 (('China', 'Beijing'), 55.202587420158),
 (('Congo', 'Brazzaville'), 78.17122244944332),
 (('Croatia', 'Zagreb'), 54.41298082378286),
 (('Cuba', 'Havana'), 75.84939520562996),
 (('Cyprus